In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
df = spark.read.format("parquet")\
        .load("abfss://bronze@databricksetee.dfs.core.windows.net/orders")

In [0]:
df.printSchema()

In [0]:
df = df.withColumnRenamed("_rescued_data", "rescued_data")

In [0]:
df = df.drop("rescued_data")

In [0]:
from pyspark.sql.functions import to_timestamp, col, year
df = df.withColumn("order_date", to_timestamp(col("order_date")))

#display(df)


In [0]:
df = df.withColumn("year", year(col("order_date")))
display(df)

In [0]:
df1 = df.withColumn("flag", dense_rank().over(Window.partitionBy("year").orderBy(col("total_amount").desc())))
#display(df1)

In [0]:
df1 = df1.withColumn("rank_flag", rank().over(Window.partitionBy("year").orderBy(col("total_amount").desc())))
display(df1)

In [0]:
df1 = df1.withColumn("row_flag", row_number().over(Window.partitionBy("year").orderBy(col("total_amount").desc())))
display(df1)

### Class OOP

In [0]:
class windows:
    def dense_rank(self, df1):
        dense_rank_df = df.withColumn("flag", dense_rank().over(Window.partitionBy("year").orderBy(col("total_amount").desc())))
        return dense_rank_df
    
    def rank(self, df):
        rank_df = df.withColumn("rank_flag", rank().over(Window.partitionBy("year").orderBy(col("total_amount").desc())))
        return rank_df
    
    def row_number(self, df):
        row_number_df = df.withColumn("row_number", rank().over(Window.partitionBy("year").orderBy(col("total_amount").desc())))
        return row_number_df
    

In [0]:
df_new = df
df_new.display()

In [0]:
obj = windows()
dense_rank_result = obj.dense_rank(df_new)
dense_rank_result.display()

### Data Writting


In [0]:
df.write.format("delta").mode("append").save("abfss://silver@databricksetee.dfs.core.windows.net/orders")


In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_cata.silver.orders_silver
USING DELTA
LOCATION 'abfss://silver@databricksetee.dfs.core.windows.net/orders/';